# Part 1: Identifying "Top Selling" Items
We define top selling items as items which had the greatest total sales over this week across all countries.

In [4]:
import pandas as pd
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv("Online Retail.csv", encoding = "ISO-8859-1")
df.shape

(541909, 8)

In [6]:
# remove cancelled invoices, which are invoices where InvoiceNo starts with the letter "c"
df['cancelled'] = df.apply(lambda row: str(row.InvoiceNo)[0] == "c", axis = 1) 
print("number of cancelled invoices: ", np.sum(df['cancelled']))

number of cancelled invoices:  0


In [7]:
# create a "data" column so we can identify best selling items for a given range
df["date"] = df.apply(lambda row: row.InvoiceDate.split(" ")[0], axis = 1) 
df["timestamp"] = df.apply(lambda x: time.mktime(datetime.datetime.strptime(x['date'], "%m/%d/%y").timetuple()), axis = 1) 
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,date,timestamp
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom,False,12/1/10,1.291180e+09
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom,False,12/1/10,1.291180e+09
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom,False,12/1/10,1.291180e+09
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom,False,12/1/10,1.291180e+09
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom,False,12/1/10,1.291180e+09


In [8]:
# we want best selling items for thanksgiving week

start = "11/27/11"
end = "12/3/11"

select = (df['timestamp'] <= time.mktime(datetime.datetime.strptime(end, "%m/%d/%y").timetuple())) & (df['timestamp'] >= time.mktime(datetime.datetime.strptime(start, "%m/%d/%y").timetuple()))
df_small = df[select]

In [9]:
# return a list of best sellers by stock id over that period
best_sellers = list(df_small.groupby("StockCode")['Quantity'].sum().sort_values(ascending=False)[:3].index)
best_sellers

['23084', '22197', '23582']

# part 2 - by country
given a stockcode id, return a dataframe of predictions by country

Build a model that predicts sales quantities for the 7 days from 11/27/2011 - 12/3/2011 Sun - Sat).

https://amueller.github.io/COMS4995-s18/slides/aml-25-042518-time-series/#43

In [10]:
cutoff = "11/27/11"

select = (df['timestamp'] <= time.mktime(datetime.datetime.strptime(cutoff, "%m/%d/%y").timetuple()))
train_df = df[select]
sorted_train_df = train_df.sort_values('timestamp')

In [11]:
test_period = ['11/27/11', '11/28/11', '11/29/11', '11/30/11', '12/1/11', '12/2/11', '12/3/11', '12/4/11', '12/5/11', '12/6/11', '12/7/11', '12/8/11', '12/9/11', '12/10/11']

In [27]:
import fbprophet
from fbprophet import Prophet
from pandas import to_datetime

def make_predictions(df, stockcode_id:str, period):
    
    # segment by Stockcode id
    df_23084 = df[df['StockCode'] == stockcode_id]

    # fit model and make predictions
    data = {time: period}
    
    countries = set(list(df_23084['Country']))
    for country in countries:
        print(">>", country, df_23084[df_23084['Country'] == country].shape)

        # ignore countries that have few sales
        if df_23084[df_23084['Country'] == country].shape[0] < 2:
            data[country] = [0] * len(period)
        else:
            # segment by country: 
            df_23084_uk = df_23084[df_23084['Country'] == country]

            # format data and fit model
            x = df_23084_uk[['date', 'Quantity']]
            x.columns = ['ds', 'y']
            model = Prophet()
            model.fit(x)

            # make predictions
            test = pd.DataFrame({"ds":period})
            predictions = model.predict(test)
            pred = {country: predictions['yhat'].values}
            data[country] = pred[country]
    return pd.DataFrame(data)

In [28]:
p = make_predictions(sorted_train_df, stockcode_id='23582', period=test_period)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


>> Italy (1, 11)
>> Channel Islands (1, 11)
>> United Kingdom (192, 11)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 2.


>> France (4, 11)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.


>> Germany (2, 11)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.


>> Portugal (2, 11)
>> Finland (1, 11)
>> Netherlands (1, 11)


In [29]:
p

,<module 'time' (built-in)>,Italy,Channel Islands,United Kingdom,France,Germany,Portugal,Finland,Netherlands
0,11/27/11,0,0,2.528192,-9.874980,10.0,5.384615,0,0
1,11/28/11,0,0,22.718173,-7.915490,10.0,3.846154,0,0
2,11/29/11,0,0,2.596520,-10.724030,10.0,2.307692,0,0
3,11/30/11,0,0,9.095039,-11.148556,10.0,0.769231,0,0
4,12/1/11,0,0,5.046209,5.389838,10.0,-0.769231,0,0
5,12/2/11,0,0,4.460591,-11.997606,10.0,-2.307692,0,0
6,12/3/11,0,0,2.842135,-12.422131,10.0,-3.846154,0,0
7,12/4/11,0,0,0.636192,-12.846657,10.0,-5.384615,0,0
8,12/5/11,0,0,20.826173,-10.887167,10.0,-6.923077,0,0
9,12/6/11,0,0,0.704519,-13.695707,10.0,-8.461538,0,0
